In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import datetime as datetime
import matplotlib.colors as mcolors
import seaborn as sns

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import interpolate
from geopack import geopack


Load IGRF coefficients ...


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
hemi = 'NH'
id = '1'

In [5]:
#ds = xr.open_dataset(f'/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT_Data/test/half_day_1deg.nc')
ds = xr.open_dataset(f'/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT_Data/test/202008_40_180_2min.nc')

def calculate_tilt_angle(ds):
    t0 = np.datetime64('1970-01-01T00:00:00')
    t1 = ds['dt']  # Access the datetime coordinate
    tdiff = (t1 - t0) / np.timedelta64(1, 's')  # Convert time difference to seconds
    tilt_angle = xr.apply_ufunc(geopack.recalc, tdiff, vectorize=True)  # Vectorized calculation
    
    return tilt_angle

ds['tilt_angle'] = calculate_tilt_angle(ds)
#ds = ds.sel(dt=slice('2023-01-01', '2024-12-31'))
ds

<xarray.Dataset> Size: 3GB
Dimensions:         (dt: 21212, lat: 40, lon: 180)
Coordinates:
  * dt              (dt) datetime64[ns] 170kB 2020-08-01 ... 2020-08-31T23:58:00
  * lon             (lon) int32 720B 1 3 5 7 9 11 13 ... 349 351 353 355 357 359
  * lat             (lat) int32 160B 50 51 52 53 54 55 56 ... 84 85 86 87 88 89
Data variables:
    pot             (dt, lat, lon) float32 611MB ...
    fac             (dt, lat, lon) float32 611MB ...
    sxx             (dt, lat, lon) float32 611MB ...
    syy             (dt, lat, lon) float32 611MB ...
    sxy             (dt, lat, lon) float32 611MB ...
    BX_GSE          (dt) float64 170kB ...
    BY_GSE          (dt) float64 170kB ...
    BZ_GSE          (dt) float64 170kB ...
    flow_speed      (dt) float64 170kB ...
    proton_density  (dt) float64 170kB ...
    tilt_angle      (dt) float64 170kB 0.2651 0.2637 0.2623 ... 0.0953 0.09394

In [6]:
def clock_angle(By, Bz):
    magnitude = np.sqrt(By**2 + Bz**2)
    cos_theta = np.arccos(Bz / magnitude)
    return cos_theta

def dayside_reconnection_rate(v_sw, By, Bz):
    By = By * 1e-9 #nT to T
    Bz = Bz * 1e-9
    Byz = np.sqrt(By**2 + Bz**2)
    v_sw = v_sw * 1e3 #km/s to m/s
    l_0 = 6371 * 1000 * 3.8 #3.8 R_E i m

    L_eff = l_0 * (v_sw/(4*10**5))**(1/3)
    theta = clock_angle(By, Bz)
    Phi_D = L_eff * v_sw * Byz * np.sin(theta/2)**(9/2)
    return Phi_D / 1e3

#ds['Phi_D'] = dayside_reconnection_rate(ds['flow_speed'], ds['BY_GSE'], ds['BZ_GSE'])
#ds

In [7]:
# Initialize scalers for the target variable and input variables
input_scaler = StandardScaler() #scale using standard dev where mean is 0 and std is 1

# Extract the target variable and reshape for scaling
target_var = ds['fac'].values  # shape (t (n), mlat (50), mlt (24))

# Extract and scale input variables (variables that are dependent only on 'dt')
input_vars = ['BY_GSE', 'BZ_GSE', 'flow_speed', 'proton_density', 'tilt_angle']
#input_vars = ['tilt_angle']
input_data = np.array([ds[var].values for var in input_vars]).T  # shape (22320, number_of_vars)
input_data_scaled = input_scaler.fit_transform(input_data)

file_path = f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_scaler_{hemi}_id{id}.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(input_scaler, file)

def create_sequences(target_data, input_data, lb):
    X, y = [], []
    for i in range(len(target_data) - lb):
        X.append(input_data[i:i+lb].T)
        y.append(target_data[i+lb])

    return np.array(X), np.array(y)

lookback = 45
X, y = create_sequences(target_var, input_data_scaled, lookback)

X.shape, y.shape

((21167, 5, 45), (21167, 40, 180))

In [8]:
def split_data(X, y, train_size=0.8, test_size=0.1):
    n = len(X)
    train_size = int(n*train_size)
    test_size = int(n*test_size)
    
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:train_size+test_size], y[train_size:train_size+test_size]
    X_val, y_val = X[train_size+test_size:], y[train_size+test_size:]
    
    return X_train, y_train, X_test, y_test, X_val, y_val

X_train, y_train, X_test, y_test, X_val, y_val = split_data(X, y)
X_train.shape, X_val.shape, X_test.shape

((16933, 5, 45), (2118, 5, 45), (2116, 5, 45))

In [9]:
# Convert data to PyTorch tensors and move to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = np.array(y_test)

# Create DataLoader
batch_size = 128
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
mlat_len = np.size(ds['lat'].values)
mlt_len = np.size(ds['lon'].values)

In [11]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, 
                            batch_first=True) 
        
        self.fc = nn.Linear(hidden_size, mlat_len*mlt_len)

    def forward(self, x):
        batch_size = x.size(0) # Get the batch size
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device) # Initial hidden state
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device) # Initial cell state
        
        out, _ = self.lstm(x, (h0, c0)) 
        out = self.fc(out[:, -1, :]) # Get the last output of the LSTM
        out = out.view(-1, mlat_len, mlt_len) # Reshape to (batch_size, mlat, mlt)
        return out
    
    def reset_states(self):
        # Reset the internal states of the LSTM layer
        self.lstm.reset_parameters()

# Instantiate the model with lookback size
model = LSTM(lookback, 128, 2)
model.to(device)

LSTM(
  (lstm): LSTM(45, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=7200, bias=True)
)

In [12]:
def calculate_delta(lat1, lon1, lat2, lon2):
    lat1 = np.abs(lat1)
    lat2 = np.abs(lat2)
    
    radius = 6371.008 * 1000

    area = radius**2 * (np.sin(np.radians(lat2)) - np.sin(np.radians(lat1))) * (np.radians(lon2) - np.radians(lon1))
    
    return area 

mlat = np.linspace(40,90,mlat_len) #50 intervals between 40° to 90°
mlt = np.linspace(0,360,mlt_len) #24 intervals between 0° to 360°

def calculate_area(mlat, mlt):
    areas = np.zeros((mlat_len-1, mlt_len-1))
    for i in range(len(mlat) - 1):
        for j in range(len(mlt) - 1):
            lat1, lat2 = mlat[i], mlat[i + 1]
            lon1, lon2 = mlt[j], mlt[j + 1]

            area = calculate_delta(lat1, lon1, lat2, lon2)
            areas[i, j] = area

    #interpolate the areas back to shape of mlat, mlt
    #Currently 49x23, interpolate back to 50x24
    x = np.arange(areas.shape[1])
    y = np.arange(areas.shape[0])
    f = interpolate.interp2d(x, y, areas, kind='linear')
    xnew = np.arange(0, areas.shape[1], areas.shape[1]/(areas.shape[1]+1))
    ynew = np.arange(0, areas.shape[0], areas.shape[0]/(areas.shape[0]+1))
    areas = f(xnew, ynew)

    return areas

areas = calculate_area(mlat, mlt)

def weighted_loss_function(y_true, y_pred):

    # Normalize the area
    weights = areas / np.min(areas)

    loss = torch.mean(torch.abs(y_true - y_pred) * torch.tensor(weights, dtype=torch.float32).to(device))
    
    return loss

/tmp/ipykernel_72200/2714849424.py:28: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f = interpolate.interp2d(x, y, areas, kind='linear')
/tmp/ipykernel_72200/2714849424.py:31: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=7e-4) # Adam optimizer
scheduler = StepLR(optimizer, step_size=8, gamma=0.7) #decay learning rate by 0.7 every 8 epochs

In [14]:
# Model settings
num_epochs = 50
train_losses = []
val_losses = []

# Early stopping
best_val_loss = float('inf')
patience = 7
counter = 0

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader:
        # Forward pass
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        train_outputs = model(X_batch)
        #loss = loss_function(train_outputs, y_batch)
        loss = weighted_loss_function(y_batch, train_outputs)

        # Backward pass and optimization
        optimizer.zero_grad() #optimize using Adam (gradient descent)
        loss.backward() #backpropagation. aka send the error back across the network
        optimizer.step() #update the weights and 'step' in the direction of the gradient
        epoch_loss += loss.item() #add the loss to the epoch loss

    # Step the scheduler
    scheduler.step() #decay the learning rate

    train_loss = epoch_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval() #set the model to evaluation mode. No weights will be updated
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            val_outputs = model(X_batch)
            #val_loss += loss_function(val_outputs, y_batch).item()
            val_loss += weighted_loss_function(y_batch, val_outputs).item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    # Early stopping
    # This will save the model with the best validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_model_{hemi}_id{id}.pt')
    else:
        counter += 1
        if counter == patience:
            print(f'Validation loss did not improve for {patience} epochs. Stopping training.')
            break

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss (MAWE): {train_loss:.4f}, Val Loss (MAWE): {val_loss:.4f}, Best Val Loss: {best_val_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.1e}')


Epoch [1/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.1768, Best Val Loss: 2.1768, LR: 7.0e-04
Epoch [2/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.1531, Best Val Loss: 2.1531, LR: 7.0e-04
Epoch [3/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.1404, Best Val Loss: 2.1404, LR: 7.0e-04
Epoch [4/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.1449, Best Val Loss: 2.1404, LR: 7.0e-04
Epoch [5/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.0925, Best Val Loss: 2.0925, LR: 7.0e-04
Epoch [6/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.0744, Best Val Loss: 2.0744, LR: 7.0e-04
Epoch [7/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.0079, Best Val Loss: 2.0079, LR: 7.0e-04
Epoch [8/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.0191, Best Val Loss: 2.0079, LR: 4.9e-04
Epoch [9/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.0327, Best Val Loss: 2.0079, LR: 4.9e-04
Epoch [10/50], Train Loss (MAWE): nan, Val Loss (MAWE): 2.0544, Best Val Loss: 2.0079, LR: 4.9e-04
Epoch [11/50], Trai

In [15]:
best_model = LSTM(lookback, 128, 2)
best_model.load_state_dict(torch.load(f'/home/sachin/Documents/NIPR/Research/Data/ML/SMRAI3/smrai3_model_{hemi}_id{id}.pt'))
best_model.to(device)

#predict every 128 samples to save cuda memory then append
batch_size = 128
predictions = []
best_model.eval()
with torch.no_grad():
    for i in range(0, len(X_test), batch_size):
        X_batch = X_test[i:i+batch_size]
        y_pred = best_model(X_batch)
        predictions.append(y_pred.cpu().numpy())
     
predictions = np.concatenate(predictions)

In [16]:
RMSE = np.sqrt(np.mean((y_test - predictions)**2))
NRMSE = RMSE / (np.max(y_test) - np.min(y_test))
MAE = np.mean(np.abs(y_test - predictions))
NMAE_std = MAE / np.std(y_test)
NMAE_mm = MAE / (np.max(y_test) - np.min(y_test))
R = np.corrcoef(y_test.flatten(), predictions.flatten())[0, 1]

def skill(m, o):
    skill = 1 - (np.sum((m - o)**2) / np.sum((o - np.mean(o))**2))
    return skill      

PE = skill(predictions, y_test)

print(f'Test Loss (MAE): {MAE:.4f}, Test Loss (NMAE_std): {NMAE_std:.4f}, R: {R:.2f}, PE: {PE:.2f}')

Test Loss (MAE): 0.0431, Test Loss (NMAE_std): 0.4103, R: 0.65, PE: 0.40


Test Loss (MAE): 0.0431, Test Loss (NMAE_std): 0.4103, R: 0.65, PE: 0.40